In [145]:
import rasterio
import numpy as np
from PIL import Image
from pyspark import SparkContext

path_to_image = '../samples/2019-02-01.tif'
profile = None

In [84]:
spark = SparkContext(appName='Reading image with RDD ')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Reading image with RDD , master=local[*]) created by __init__ at /var/folders/sp/2my_wkd16kz0bb6sfd_yyryr0000gn/T/ipykernel_4849/2196386975.py:1 

In [171]:
def read_image(path):
    with rasterio.open(path) as dataset:
        global profile
        data = dataset.read()
        profile = dataset.profile
    data = data.transpose((1, 2, 0))
    print(data.shape)
    return data
    # data = np.squeeze(data, axis=1)
    # return data

In [172]:
rdd_image = spark.parallelize([path_to_image]).map(read_image)

In [175]:
foo = rdd_image.collect()[0]

(152, 122, 16)


array([[63976., 64002., 64002., ..., 63884., 63884., 63858.],
       [63976., 64002., 64002., ..., 63884., 63884., 63858.],
       [63976., 64002., 64002., ..., 63884., 63884., 63858.],
       ...,
       [61616., 61708., 61708., ..., 61485., 61485., 61459.],
       [61616., 61708., 61708., ..., 61485., 61485., 61459.],
       [62285., 62462., 62462., ..., 61459., 61459., 61367.]])

In [232]:
def calculate_ndvi(image):
    red_band = image[:, :, 3].astype(float)
    infra_band = image[:, :, 7].astype(float)
    return (infra_band - red_band) / (infra_band + red_band)

In [233]:
def inversion_effect(image):
    global profile
    ndvi = calculate_ndvi(image)
    profile['width'] = ndvi.shape[1]
    profile['height'] = ndvi.shape[0]
    profile['dtype'] = ndvi.dtype
    profile['count'] = 1
    print(ndvi.shape)
    print(profile)
    with rasterio.open('../samples/foo.tiff', 'w', **profile) as dst:
        dst.write(ndvi, indexes=profile['count'])

In [234]:
rdd_processed = rdd_image.map(inversion_effect)

In [235]:
result_of_process = rdd_processed.collect()

(152, 122, 16)
(152, 122)
{'driver': 'GTiff', 'dtype': dtype('float64'), 'nodata': None, 'width': 122, 'height': 152, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(9.220090163928715e-05, 0.0, -56.20873019,
       0.0, -8.994572368421818e-05, -12.21358084), 'blockysize': 8, 'tiled': False, 'compress': 'deflate', 'interleave': 'pixel'}


In [21]:
result_of_process

[None]